In [0]:
#加载google drive上的数据集
!echo 'Installing required software'
!apt-get install -y -qq software-properties-common module-init-tools 2>&1 > /dev/null
!echo 'Add apt-repository with Google.Drive Fuse'
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!echo 'Updating packages...'
!apt-get update -y -qq
!echo 'Installing google-drive-ocamlfuse fuse...'
!apt-get install -y -qq google-drive-ocamlfuse fuse
!echo 'Authenticate Fuse in Google.Drive...'
from google.colab import auth
from oauth2client.client import GoogleCredentials
import getpass
auth.authenticate_user()
creds = GoogleCredentials.get_application_default()
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass('Enter auth code here: ')
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
!echo 'Authenticated!'
!echo 'Creating mount directory'
!mkdir ./drive2
!echo 'Mounting...'
!google-drive-ocamlfuse ./drive2
!ls ./drive2
!echo 'Mounted!'

Installing required software
Add apt-repository with Google.Drive Fuse
Updating packages...
Installing google-drive-ocamlfuse fuse...
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 145118 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.17-0ubuntu2~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.17-0ubuntu2~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.17-0ubuntu2~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Authenticate Fuse in Google.Drive...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Enter auth code here: ··········
Please, open the following URL in a web browser: https://accounts.googl

In [0]:
!ls

adc.json  drive2  sample_data


In [0]:
!pip uninstall lightgbm -y

Uninstalling lightgbm-2.2.3:
  Successfully uninstalled lightgbm-2.2.3


In [0]:
!pip install lightgbm==2.2.3 --install-option=--gpu

/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
     |████████████████████████████████| 655kB 2.8MB/s 
Skipping wheel build for lightgbm, due to binaries being disabled for it.
    Running setup.py install for lightgbm ... done


In [0]:
!ls

adc.json  drive2  sample_data


In [0]:
#解压数据集
!unzip -o ./drive2/Colab\ Notebooks/hy_round2_train_20200225.zip

流式输出内容被截断，只能显示最后 5000 行内容。
  inflating: hy_round2_train_20200225/23167.csv  
  inflating: hy_round2_train_20200225/23168.csv  
  inflating: hy_round2_train_20200225/23169.csv  
  inflating: hy_round2_train_20200225/23170.csv  
  inflating: hy_round2_train_20200225/23171.csv  
  inflating: hy_round2_train_20200225/23172.csv  
  inflating: hy_round2_train_20200225/23173.csv  
  inflating: hy_round2_train_20200225/23174.csv  
  inflating: hy_round2_train_20200225/23175.csv  
  inflating: hy_round2_train_20200225/23176.csv  
  inflating: hy_round2_train_20200225/23177.csv  
  inflating: hy_round2_train_20200225/23178.csv  
  inflating: hy_round2_train_20200225/23179.csv  
  inflating: hy_round2_train_20200225/23180.csv  
  inflating: hy_round2_train_20200225/23181.csv  
  inflating: hy_round2_train_20200225/23182.csv  
  inflating: hy_round2_train_20200225/23183.csv  
  inflating: hy_round2_train_20200225/23184.csv  
  inflating: hy_round2_train_20200225/23185.csv  
  inflating: hy_round2_

In [0]:
import pandas as pd
import numpy as np
import multiprocessing 
from datetime import datetime
import os
import math
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.preprocessing import KBinsDiscretizer

In [0]:
#汉明距离
def haversine_array(lat1, lng1, lat2, lng2):
     lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2))
     AVG_EARTH_RADIUS = 6371 # in km
     lat = lat2 - lat1
     lng = lng2 - lng1
     d = np.sin(lat * 0.5) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(lng * 0.5) ** 2
     h = 2 * AVG_EARTH_RADIUS * np.arcsin(np.sqrt(d))
     return h
#隐曼哈顿距离
def dummy_manhattan_distance(lat1, lng1, lat2, lng2):
     a = haversine_array(lat1, lng1, lat1, lng2)
     b = haversine_array(lat1, lng1, lat2, lng1)
     return a + b
#
def bearing_array(lat1, lng1, lat2, lng2):
     lng_delta_rad = np.radians(lng2 - lng1)
     lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2))
     y = np.sin(lng_delta_rad) * np.cos(lat2)
     x = np.cos(lat1) * np.sin(lat2) - np.sin(lat1) * np.cos(lat2) * np.cos(lng_delta_rad)
     return np.degrees(np.arctan2(y, x))
    

In [0]:
#构造统计特征
#返回值是个字典
def stat_feature(data_):
    result_ = {}
    for feature in data_.columns:
      if(feature in {'渔船ID','time','type'}):
        continue
      data_min = data_[feature].min()
      data_max = data_[feature].max()
      data_mean = data_[feature].mean()
      data_std = data_[feature].std()
      data_skew = data_[feature].skew()
      data_kurt = data_[feature].kurt()
      #均方根？？
      data_rms=math.sqrt(pow(data_mean,2) + pow(data_std,2))
      #波形因子
      data_boxing=data_rms / ((abs(data_[feature]).mean())+0.0001)
      #峰值因子
      #print('数据长度为'+str(len(data_[feature])))
      data_fengzhi=(max(data_[feature])) / (data_rms+0.00001)
      #脉冲因子
      data_maichong=(max(data_[feature])) / ((abs(data_[feature]).mean())+0.00001)
      #裕度因子
      data_yudu=(max(data_[feature])) / (pow((data_[feature].map(lambda x: math.sqrt(abs(x))).sum()/(len(data_))),2)+0.00001)
      data_range = data_max - data_min
      data_covar = data_std/(data_mean+0.0001)
      #data_cid_ce = cid_ce(data_[feature])
      #data_longest_above_median = longest_above_median(data_[feature])
      result__ = {
              feature+'_min': data_min,
              feature+'_max': data_max,
              feature+'_mean':data_mean,
              feature+'_std':data_std,
              feature+'_skew':data_skew,
              feature+'_kurt':data_kurt,
              feature+'_range':data_range,
              feature+'_covar':data_covar,
              feature+'_rms':data_rms,
              feature+'_boxing':data_boxing,
              feature+'_fengzhi':data_fengzhi,
              feature+'_maichong':data_maichong,
              feature+'_yudu':data_yudu,
              #feature+'_cid_ce':data_cid_ce,
              #feature+'_longest_above_median':data_longest_above_median,
                  }
      result_.update(result__)
    return result_

In [0]:
def construct_newcol(data):
  data['lat_var'] = data['lat'].diff()
  #坐标y的变化
  data['lon_var'] = data['lon'].diff()
  #距离的变化
  data['distance_var'] = np.sqrt(data['lat_var'].values*data['lat_var'].values + data['lon_var'].values*data['lon_var'].values)
  #速度的变化
  data['velocity_var'] = data['速度'].diff()
  #角度的变化
  data['angle_var'] = data['方向'].diff()
  #计算出来的角度
  data['computed_angle'] = np.arctan(data['lon_var'].values/(data['lat_var'].values+0.00001))
  #计算坐标计算出来的角速度
  data['computed_angle_var'] = data['computed_angle'].diff() 
  data['haversine'] = haversine_array(data['lat'].shift(-1).values,
                                     data['lon'].shift(-1).values,
                                     data['lat'].values,data['lon'].values)
  data['dummy_manhattan_distance'] = dummy_manhattan_distance(data['lat'].shift(-1).values,
                                                              data['lon'].shift(-1).values,
                                                              data['lat'].values,
                                                              data['lon'].values)
  data['bearing'] = bearing_array(data['lat'].shift(-1).values,
                                  data['lon'].shift(-1).values,
                                  data['lat'].values,data['lon'].values)
  data.dropna(inplace=True)
  return data

In [0]:
def preprocessing(data,id):
  #数据构造新列
  data = construct_newcol(data)
  #数据对齐
  result = stat_feature(data)
  for i in [0.25,0.75]:
    result['lat_quantile_'+str(i)] = data['lat'].quantile(i)
    result['lon_quantile_'+str(i)] = data['lon'].quantile(i)
    result['速度_quantile_'+str(i)] = data['速度'].quantile(i)
    result['distance_var_quantile_'+str(i)] = data['distance_var'].quantile(i)
    result['velocity_var_quantile_'+str(i)] = data['velocity_var'].quantile(i)
    result['angle_var_quantile_'+str(i)] = data['angle_var'].quantile(i)
    
  
  
  result['latlon_cov'] = data['lat'].cov(data['lon'])
  #数据坐标信息错误的话 这个值应该会不一样
  result['distance_var_velocity_cov'] = data['distance_var'].cov(data['速度'])
  #数据坐标信息错误的话 这个值应该也会不一样
  result['angle_var_computed_angle_var_cov'] = data['angle_var'].cov(data['computed_angle_var'])
  #计算角速度速度的协方差
  result['angle_var_velocity_cov'] = data['angle_var'].cov(data['速度'])
  #计算角速度，加速度协方差
  result['angle_var_velocity_var_cov'] = data['angle_var'].cov(data['velocity_var']) 
  #result['xy_cov'] = data['x'].cov(data['y'])
  result['max_area'] = (result['lat_max'] - result['lat_min'])*(result['lon_max']-result['lon_min'])
  result['circle_area'] = pow(result['lat_max'] - result['lat_min'] + result['lon_max']-result['lon_min'],2)
  result['line_len'] = result['lat_max'] - result['lat_min'] + result['lon_max']-result['lon_min']
  #result['xy_area'] = (result['x_max'] - result['x_min'])*(result['y_max']-result['y_min'])   
  #速度最多的值
  result['velocity_first_counts'] = pd.cut(data['速度'].values,np.linspace(0,50,51)).value_counts().index[0].left
  #速度第二多的值
  result['velocity_second_counts'] = pd.cut(data['速度'].values,np.linspace(0,50,51)).value_counts().index[1].left
  #速度第三多的值
  result['velocity_third_counts'] = pd.cut(data['速度'],np.linspace(0,50,51)).value_counts().index[2].left
  #构造直方图离散化计数特征
  result['velocity_count_rate_1'] = len(data[data['速度']<1])/len(data)
  result['velocity_count_rate_2'] = len(data[(data['速度']<2) & (data['速度']>1)])/len(data)
  result['velocity_count_rate_3'] = len(data[(data['速度']<5) & (data['速度']>2)])/len(data)
  result['velocity_count_rate_4'] = len(data[(data['速度']<7) & (data['速度']>5)])/len(data)
  result['velocity_count_rate_5'] = len(data[(data['速度']<10) & (data['速度']>7)])/len(data)
  result['velocity_count_rate_6'] = len( data[data['速度']>10])/len(data)  
  #当时的时间
  result['hour'] = np.array(data.index.hour + data.index.minute/60.0).mean()
  #result['hour_std'] = np.array(data.index.hour + data.index.minute/60.0).std()  
  result['id'] = id
  return result


In [0]:
def new_cols(data):
  result = data
  result['lat_boxing_lat_min_mul'] = data['lat_boxing'].values * data['lat_min'].values
  result['lon_boxing_lat_min_mul'] = data['lon_boxing'].values * data['lat_min'].values
  result['lat_quantile_0.25_lat_min_mul'] = data['lat_quantile_0.25'].values * data['lat_min'].values
  result['lat_boxing_lat_max_mul'] = data['lat_boxing'].values * data['lat_max'].values
  result['lat_fengzhi_lat_max_mul'] = data['lat_fengzhi'].values * data['lat_max'].values
  result['lon_boxing_lat_max_mul'] = data['lon_boxing'].values * data['lat_max'].values
  result['lon_min_lat_boxing_mul'] = data['lon_min'].values * data['lat_boxing'].values
  result['lon_max_lat_boxing_mul'] = data['lon_max'].values * data['lat_boxing'].values
  result['lon_boxing_lat_boxing_mul'] = data['lon_boxing'].values * data['lat_boxing'].values
  result['lon_max_lat_fengzhi_mul'] = data['lon_max'].values * data['lat_fengzhi'].values
  result['lon_boxing_lon_min_mul'] = data['lon_boxing'].values * data['lon_min']
  result['lon_quantile_0.25_lon_min_mul'] = data['lon_quantile_0.25'].values * data['lon_min'].values
  result['lon_boxing_lon_max_mul'] = data['lon_boxing'].values * data['lon_max'].values
  result['lon_quantile_0.25_lon_boxing_mul'] = data['lon_quantile_0.25'].values * data['lon_boxing'].values

  result['lon_min_div_lat_boxing'] = data['lon_min'].values/(data['lat_boxing'].values + 0.0000000001)
  result['lon_min_div_lon_boxing'] = data['lon_min'].values/(data['lon_boxing'].values + 0.0000000001)
  result['lon_max_div_lat_boxing'] = data['lon_max'].values/(data['lat_boxing'].values + 0.0000000001)
  result['lon_max_div_lon_boxing'] = data['lon_max'].values/(data['lon_boxing'].values + 0.0000000001)
  result['lon_boxing_div_lon_max'] = data['lon_boxing'].values/(data['lon_max'].values + 0.0000000001)
  result['lon_quantile_0.25_div_lat_boxing'] = data['lon_quantile_0.25'].values/(data['lat_boxing'].values + 0.0000000001)
  return result

In [0]:
'''
如果要改成多线程处理，逻辑上看的话
首先将需要处理的文件列出来
然后多线程处理每个文件
'''
def read_csv_train(filename):
        data_ = pd.read_csv(filename,index_col='time',parse_dates=True)
        data_.index = [datetime.strptime(i, '%m%d %H:%M:%S') for i in data_.index]
        data_type = data_['type'][0]
        #data_= data_.resample('5T').mean().interpolate()
        return data_,data_type


#读取数据集，记录不合并
def read_data(data_dir_pre):     
  data_dir = os.listdir(data_dir_pre)
  #进程数
  processnum = 16
  #读取文件
  data = {}
  proced_data = []
  count = 0
  data_type = {}
  '''
  读取文件单进程
  '''

  for filename in data_dir[:30]:
    count = count + 1
    if(count%10==0):
      print('正在读取第{}个文件'.format(count))
    data[filename.split('.',1)[0]] , data_type[filename.split('.',1)[0]] = read_csv_train(data_dir_pre + filename)
    #对数据进行插值
    data[filename.split('.',1)[0]] = data[filename.split('.',1)[0]].resample('5T').mean().interpolate()
    data[filename.split('.',1)[0]] = construct_newcol(data[filename.split('.',1)[0]])
    
  #将数据进行分箱统计
  merged_data = pd.concat(data,axis=0)
  merged_data = merged_data[merged_data.columns.drop(['渔船ID'])]
  #每个特征聚成三类
  #est_kbin = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='kmeans')
  #est_kbin.fit(merged_data)


  #预处理数据
  pool1 = multiprocessing.Pool(processes = processnum)
  data_async = []
  count = 0
  for i in data.keys():
    count = count+1
    #print('正在为数据的第{}个文件构造数据的特征'.format(count))
    if(count%10==0):
      print('正在为数据的第{}个文件构造数据的特征'.format(count))
    data_len = len(data[i])
    data_start = 0
    step = 50
    while(data_len>=step):
        #proc_ = pool1.apply_async(func =preprocessing,args = (data[i][data_start:(data_start+step)].copy(),i,est_kbin))
        proc_ = pool1.apply_async(func =preprocessing,args = (data[i][data_start:(data_start+step)].copy(),i))
        #data_async元素是元组，proc_为异步执行结果，data_type为y值
        data_async.append((proc_,data_type[i]))
        data_len = data_len -step
        data_start = data_start +step
  pool1.close()
  pool1.join()
  proced_data = [x[0].get() for x in data_async]
  data_type = [x[1] for x in data_async]
  for i in range(len(data_async)):
    proced_data[i]['type'] = data_type[i]
  #return proced_data,est_kbin
  return proced_data

In [0]:
def read_csv_test(filename):
        data_ = pd.read_csv(filename,index_col='time',parse_dates=True)
        data_.index = [datetime.strptime(i, '%m%d %H:%M:%S') for i in data_.index]
        #data_= data_.resample('5T').mean().interpolate()
        return data_

#读取数据集，记录不合并
#def read_test(data_dir_pre,est_kbin):
def read_test(data_dir_pre):
  data_dir = os.listdir(data_dir_pre)
  #进程数
  processnum = 16
  #读取文件
  data = {}
  proced_data = []
  count = 0
  '''
  读取文件单进程
  '''
 
  for filename in data_dir:
    if(count%10==0):
      print('正在读取第{}个文件'.format(count))
    data[filename.split('.',1)[0]]= read_csv_test(data_dir_pre + filename)
    #对数据进行插值操作
    data[filename.split('.',1)[0]] = data[filename.split('.',1)[0]].resample('5T').mean().interpolate()
    #数据划分完成，都放入data字典中了
    
    #构造新的列
    data[filename.split('.',1)[0]] = construct_newcol(data[filename.split('.',1)[0]])

  #预处理数据
  pool2 = multiprocessing.Pool(processes = processnum)
  data_async = []
  count = 0
  for i in data.keys():
    count = count+1
    print('正在为数据的第{}个文件构造数据的特征'.format(count))
    if(count%10==0):
      print('正在为数据的第{}个文件构造数据的特征'.format(count))
    data_len = len(data[i])
    data_start = 0
    step = 50
    if(data_len<step):
        #proc_ = pool2.apply_async(func =preprocessing,args = (data[i][data_start:(data_start+step)].copy(),i,est_kbin))
        proc_ = pool2.apply_async(func =preprocessing,args = (data[i][data_start:(data_start+step)].copy(),i))
        data_async.append(proc_)
        data_len = data_len -step
        data_start = data_start +step
    while(data_len>=step):
        #proc_ = pool2.apply_async(func = preprocessing,args = (data[i][data_start:(data_start+step)].copy(),i,est_kbin))
        proc_ = pool2.apply_async(func =preprocessing,args = (data[i][data_start:(data_start+step)].copy(),i))
        data_async.append(proc_)
        data_len = data_len -step
        data_start = data_start +step
  pool2.close()
  pool2.join()
  proced_data = [x.get() for x in data_async]
  return proced_data


In [0]:
mkdir hy_round2_test_20200225

In [0]:
!mv hy_round2_train_20200225/28* hy_round2_test_20200225/

In [0]:
def train_data():
    train_ = read_data('./hy_round2_train_20200225/')
    train_x = pd.DataFrame.from_dict(train_)
    train_y = train_x.pop('type')
    #测试得到结果需要id 
    #训练模型不需要
    #train_id = train_x.pop('id')   
    return train_x,train_y

def train(x,y):
    cw = list(class_weight.compute_class_weight('balanced',np.unique(y),y))
    lr = LogisticRegressionCV(cv=5,class_weight='balanced',scoring='f1_macro',verbose=10,random_state=0)  
    clf1 = lgb.LGBMClassifier(objective = 'multi:softmax',
                 n_estimators=900,#900
                 max_depth = 11,#8
                 num_leaves = 90,#90
                 learning_rate = 0.17,
                 feature_fraction = 0.7,
                 min_child_samples=5,
                 min_child_weight=0.001,
                 bagging_fraction = 1,
                 bagging_freq = 0,
                 reg_alpha = 0.015,
                 reg_lambda = 0,
                 cat_smooth = 0,
                 device= 'gpu',
                 gpu_platform_id= 1,
                 gpu_device_id= 0,
                 class_weight='balanced',
                 random_state=0,
                 n_jobs=-1
                )
    #听说lightgbm里的 随机森林比sklearn要快？
    clf2 = RandomForestClassifier(n_estimators = 1000,random_state=0,n_jobs=-1,class_weight='balanced')
    clf3 = CatBoostClassifier( 
                iterations=2000,
                verbose=400,
                early_stopping_rounds=200,
                task_type='GPU',
                #border_count=254,
                loss_function='MultiClass',
                class_weights=cw,
                depth=8,
                l2_leaf_reg=0.06,
                random_strength=0.01,
                random_state=0)
    clf = StackingClassifier(
                estimators=[('lgb',clf1),('rf',clf2),('catboost',clf3)],
                cv=5,
                final_estimator=lr,
                stack_method='predict_proba',
                verbose=10,
                n_jobs=1)
    
    clf.fit(x, y)
    return clf

In [0]:
#est_bin是做了每个特征dbscan分箱
train_x,train_y = train_data()

正在读取第10个文件
正在读取第20个文件
正在读取第30个文件
正在为数据的第10个文件构造数据的特征
正在为数据的第20个文件构造数据的特征
正在为数据的第30个文件构造数据的特征


KeyError: ignored

In [0]:
train_x = new_cols(train_x)

In [0]:
train = pd.concat([train_x,train_y],axis=1)

In [0]:
train.to_csv('./drive2/Colab Notebooks/train.csv',index=None)

In [0]:
train = pd.read_csv('./drive2/Colab Notebooks/train.csv')

In [0]:
#train_id = train.pop('id')

In [0]:
train_y = train.pop('type')

In [0]:
train_x = train

In [0]:
cols_for_derive = ['lat_min', 'lat_max', 'lat_mean', 'lat_std', 'lat_skew', 'lat_kurt',
       'lat_covar', 'lat_boxing', 'lat_fengzhi', 'lon_min', 'lon_max',
       'lon_mean', 'lon_skew', 'lon_covar', 'lon_boxing', '速度_std', '速度_skew',
       '速度_kurt', '速度_covar', '速度_fengzhi', '方向_max', '方向_std', '方向_kurt',
       'velocity_var_mean', 'velocity_var_skew', 'velocity_var_kurt',
       'velocity_var_covar', 'velocity_var_boxing', 'velocity_var_fengzhi',
       'velocity_var_maichong', 'velocity_var_yudu', 'angle_var_min',
       'angle_var_max', 'angle_var_mean', 'angle_var_skew', 'angle_var_kurt',
       'angle_var_covar', 'angle_var_boxing', 'angle_var_fengzhi',
       'lat_quantile_0.25', 'lon_quantile_0.25', 'velocity_var_quantile_0.25',
       'angle_var_quantile_0.25', 'lat_quantile_0.75', 'lon_quantile_0.75',
       '速度_quantile_0.75', 'velocity_var_quantile_0.75',
       'angle_var_quantile_0.75', 'latlon_cov',
       'angle_var_computed_angle_var_cov', 'angle_var_velocity_cov',
       'angle_var_velocity_var_cov', 'hour']

In [0]:
new_div_cols = pd.DataFrame()

In [0]:
for col1 in cols_for_derive:
  for col2 in cols_for_derive:
    if(col1!=col2):
      new_div_cols[col1+'_div_'+col2] = train_x[col1].values/(train_x[col2].values +0.00000001)
      print(col1 + ' dividedby '+ col2)


lat_min dividedby lat_max
lat_min dividedby lat_mean
lat_min dividedby lat_std
lat_min dividedby lat_skew
lat_min dividedby lat_kurt
lat_min dividedby lat_covar
lat_min dividedby lat_boxing
lat_min dividedby lat_fengzhi
lat_min dividedby lon_min
lat_min dividedby lon_max
lat_min dividedby lon_mean
lat_min dividedby lon_skew
lat_min dividedby lon_covar
lat_min dividedby lon_boxing
lat_min dividedby 速度_std
lat_min dividedby 速度_skew
lat_min dividedby 速度_kurt
lat_min dividedby 速度_covar
lat_min dividedby 速度_fengzhi
lat_min dividedby 方向_max
lat_min dividedby 方向_std
lat_min dividedby 方向_kurt
lat_min dividedby velocity_var_mean
lat_min dividedby velocity_var_skew
lat_min dividedby velocity_var_kurt
lat_min dividedby velocity_var_covar
lat_min dividedby velocity_var_boxing
lat_min dividedby velocity_var_fengzhi
lat_min dividedby velocity_var_maichong
lat_min dividedby velocity_var_yudu
lat_min dividedby angle_var_min
lat_min dividedby angle_var_max
lat_min dividedby angle_var_mean
lat_min divid

In [0]:
import lightgbm as lgb

In [0]:
clf2  = lgb.LGBMClassifier(objective = 'multi:softmax',
                         n_estimators=900,
                         max_depth = 11,
                         num_leaves = 90,
                         learning_rate = 0.27,
                         feature_fraction = 0.7,
                         min_child_samples=5,
                         min_child_weight=0.001,
                         bagging_fraction = 1,
                         bagging_freq = 0,
                         reg_alpha = 0.015,
                         reg_lambda = 0,
                         cat_smooth = 0,
                         device= 'gpu',
                         silent=False,
                         gpu_platform_id= 1,
                         gpu_device_id= 0,
                         class_weight='balanced',
                         verbose=10
                        )

In [0]:
clf2.fit(new_div_cols,train_y)

LGBMClassifier(bagging_fraction=1, bagging_freq=0, boosting_type='gbdt',
               cat_smooth=0, class_weight='balanced', colsample_bytree=1.0,
               device='gpu', feature_fraction=0.7, gpu_device_id=0,
               gpu_platform_id=1, importance_type='split', learning_rate=0.27,
               max_depth=11, min_child_samples=5, min_child_weight=0.001,
               min_split_gain=0.0, n_estimators=900, n_jobs=-1, num_leaves=90,
               objective='multi:softmax', random_state=None, reg_alpha=0.015,
               reg_lambda=0, silent=False, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0, verbose=10)

In [0]:
clf2.feature_importances_

array([ 74,  83,   8, ..., 157, 116, 122])

In [0]:
new_div_cols.columns[clf2.feature_importances_>490]

Index(['lon_min_div_lat_boxing', 'lon_min_div_lon_boxing',
       'lon_max_div_lat_boxing', 'lon_max_div_lon_boxing',
       'lon_boxing_div_lon_max', 'lon_quantile_0.25_div_lat_boxing'],
      dtype='object')

In [0]:
!pip install catboost


     |████████████████████████████████| 64.4MB 47kB/s 


In [0]:
from sklearn.model_selection import GridSearchCV

In [0]:
from catboost import CatBoostClassifier
from sklearn.utils import class_weight

In [0]:
cw = class_weight.compute_class_weight(class_weight='balanced',classes=np.unique(train_y),y=train_y)

In [0]:
parameters = {   
  'depth':[9]
     
}
gbm = CatBoostClassifier(iterations=2000,verbose=400,early_stopping_rounds=200,task_type='GPU',
                         border_count=254,
                    loss_function='MultiClass',
                         class_weights=cw,depth=9,l2_leaf_reg=0.06,random_strength=0.01)
gsearch = GridSearchCV(estimator=gbm, param_grid=parameters, scoring='f1_macro', cv=5, verbose=10, n_jobs=1)
gsearch.fit(train_x, train_y)
print('参数的最佳取值:{0}'.format(gsearch.best_params_))
print('最佳模型得分:{0}'.format(gsearch.best_score_))
print(gsearch.cv_results_['mean_test_score'])
print(gsearch.cv_results_['params'])

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] depth=9 .........................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0:	learn: 1.0745463	total: 39.8ms	remaining: 1m 19s
400:	learn: 0.3861803	total: 12.5s	remaining: 50s
800:	learn: 0.3038437	total: 26s	remaining: 38.9s
1200:	learn: 0.2566333	total: 39.9s	remaining: 26.5s
1600:	learn: 0.2225926	total: 53.8s	remaining: 13.4s
1999:	learn: 0.1961639	total: 1m 8s	remaining: 0us
[CV] ............................. depth=9, score=0.878, total= 2.0min
[CV] depth=9 .........................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.0min remaining:    0.0s


0:	learn: 1.0744799	total: 36.6ms	remaining: 1m 13s
400:	learn: 0.3878325	total: 12.7s	remaining: 50.7s
800:	learn: 0.3048300	total: 26.2s	remaining: 39.3s
1200:	learn: 0.2578254	total: 40.1s	remaining: 26.7s
1600:	learn: 0.2238753	total: 54.3s	remaining: 13.5s
1999:	learn: 0.1968503	total: 1m 8s	remaining: 0us
[CV] ............................. depth=9, score=0.878, total= 1.2min
[CV] depth=9 .........................................................


...

0:	learn: 1.0742993	total: 38.4ms	remaining: 1m 16s
400:	learn: 0.3852047	total: 12.7s	remaining: 50.6s
800:	learn: 0.3036296	total: 25.9s	remaining: 38.7s
1200:	learn: 0.2576817	total: 39.4s	remaining: 26.2s
1600:	learn: 0.2239688	total: 53.1s	remaining: 13.2s
1999:	learn: 0.1973865	total: 1m 7s	remaining: 0us
[CV] ............................. depth=9, score=0.876, total= 1.2min
[CV] depth=9 .........................................................


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:  5.6min remaining:    0.0s


0:	learn: 1.0751588	total: 38.7ms	remaining: 1m 17s
400:	learn: 0.3879716	total: 12.6s	remaining: 50.1s
800:	learn: 0.3047902	total: 26s	remaining: 38.9s
1200:	learn: 0.2584238	total: 39.7s	remaining: 26.4s
1600:	learn: 0.2254015	total: 53.6s	remaining: 13.4s
1999:	learn: 0.1989489	total: 1m 7s	remaining: 0us
[CV] ............................. depth=9, score=0.873, total= 1.2min


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  6.8min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  6.8min finished


0:	learn: 1.0744961	total: 39.9ms	remaining: 1m 19s
400:	learn: 0.3921705	total: 12.9s	remaining: 51.4s
800:	learn: 0.3120748	total: 26.6s	remaining: 39.9s
1200:	learn: 0.2684162	total: 40.7s	remaining: 27.1s
1600:	learn: 0.2361483	total: 55.1s	remaining: 13.7s
1999:	learn: 0.2110002	total: 1m 9s	remaining: 0us
参数的最佳取值:{'depth': 9}
最佳模型得分:0.8761763101445463
[0.87617631]
[{'depth': 9}]


In [0]:
pip install Boruta

     |████████████████████████████████| 61kB 1.9MB/s 


In [0]:
from boruta import BorutaPy

In [0]:
rf = RandomForestClassifier(n_estimators=1000,max_depth=6,random_state=0,n_jobs=-1,class_weight='balanced',verbose=5)

In [0]:
feat_selector = BorutaPy(rf, n_estimators='auto', verbose=2, random_state=0,max_iter=100)

In [0]:
feat_selector.fit(train_x.to_numpy(), train_y.to_numpy())

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


building tree 2 of 353building tree 3 of 353
building tree 4 of 353

building tree 1 of 353
building tree 5 of 353
building tree 6 of 353
building tree 7 of 353
building tree 8 of 353
building tree 9 of 353
building tree 10 of 353
building tree 11 of 353
building tree 12 of 353
building tree 13 of 353
building tree 14 of 353
building tree 15 of 353


[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    8.1s


building tree 16 of 353
building tree 17 of 353
building tree 18 of 353
building tree 19 of 353
building tree 20 of 353
building tree 21 of 353
building tree 22 of 353
building tree 23 of 353
building tree 24 of 353
building tree 25 of 353
building tree 26 of 353
building tree 27 of 353
building tree 28 of 353
building tree 29 of 353
building tree 30 of 353
building tree 31 of 353
building tree 32 of 353
building tree 33 of 353
building tree 34 of 353
building tree 35 of 353
building tree 36 of 353
building tree 37 of 353
building tree 38 of 353
building tree 39 of 353
building tree 40 of 353
building tree 41 of 353
building tree 42 of 353
building tree 43 of 353
building tree 44 of 353
building tree 45 of 353
building tree 46 of 353
building tree 47 of 353
building tree 48 of 353
building tree 49 of 353
building tree 50 of 353
building tree 51 of 353
building tree 52 of 353
building tree 53 of 353
building tree 54 of 353
building tree 55 of 353
building tree 56 of 353
building tree 57

[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:   44.7s


...

[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed:  3.1min


building tree 284 of 349
building tree 285 of 349
building tree 286 of 349
building tree 287 of 349
building tree 288 of 349
building tree 289 of 349
building tree 290 of 349
building tree 291 of 349
building tree 292 of 349
building tree 293 of 349
building tree 294 of 349
building tree 295 of 349
building tree 296 of 349
building tree 297 of 349
building tree 298 of 349
building tree 299 of 349
building tree 300 of 349
building tree 301 of 349
building tree 302 of 349
building tree 303 of 349
building tree 304 of 349
building tree 305 of 349
building tree 306 of 349
building tree 307 of 349
building tree 308 of 349
building tree 309 of 349
building tree 310 of 349
building tree 311 of 349
building tree 312 of 349
building tree 313 of 349building tree 314 of 349

building tree 315 of 349
building tree 316 of 349
building tree 317 of 349
building tree 318 of 349
building tree 319 of 349
building tree 320 of 349
building tree 321 of 349
building tree 322 of 349
building tree 323 of 349


[Parallel(n_jobs=-1)]: Done 349 out of 349 | elapsed:  3.9min finished


Iteration: 	99 / 100
Confirmed: 	215
Tentative: 	5
Rejected: 	5


BorutaPy finished running.

Iteration: 	100 / 100
Confirmed: 	215
Tentative: 	3
Rejected: 	5


BorutaPy(alpha=0.05,
         estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                          class_weight='balanced',
                                          criterion='gini', max_depth=6,
                                          max_features='auto',
                                          max_leaf_nodes=None, max_samples=None,
                                          min_impurity_decrease=0.0,
                                          min_impurity_split=None,
                                          min_samples_leaf=1,
                                          min_samples_split=2,
                                          min_weight_fraction_leaf=0.0,
                                          n_estimators=349, n_jobs=-1,
                                          oob_score=False,
                                          random_state=RandomState(MT19937) at 0x7F88D33A6360,
                                          verbose=5, warm_start=

In [0]:
# check selected features - first 5 features are selected
mask = [not x for x in feat_selector.support_]





In [0]:
len(train_x.columns)

225

In [0]:
train_x.columns[mask]

Index(['lat_var_skew', 'lon_var_skew', 'velocity_var_skew',
       'velocity_var_covar', 'angle_var_mean', 'computed_angle_var_mean',
       'computed_angle_var_skew', 'computed_angle_var_covar',
       'angle_var_velocity_cov', 'hour_std'],
      dtype='object')

In [0]:
# check ranking of features
feat_selector.ranking_

225

In [0]:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     # call transform() on X to filter it down to selected features
X_filtered = feat_selector.transform(train_x.to_numpy())                                                                                           

In [0]:
X_filtered

array([[ 2.59030000e+01,  2.59040000e+01,  2.59037447e+01, ...,
         1.02527540e+00,  7.07269699e-01,  1.55626998e+00],
       [ 2.57690000e+01,  2.59040000e+01,  2.58464043e+01, ...,
         2.13970791e+00,  6.58542956e-01,  8.59576340e-01],
       [ 2.57710000e+01,  2.58130000e+01,  2.57919362e+01, ...,
         1.58300912e+00,  3.60241515e-01,  8.87063348e-01],
       ...,
       [ 2.67980000e+01,  2.67980000e+01,  2.67980000e+01, ...,
        -9.99000000e+02,  4.59624143e-01,  1.33722520e+00],
       [ 2.67980000e+01,  2.67980000e+01,  2.67980000e+01, ...,
        -9.99000000e+02,  4.88097603e-01,  8.07673463e-01],
       [ 2.67980000e+01,  2.67980000e+01,  2.67980000e+01, ...,
        -9.99000000e+02,  3.52469631e-01,  1.31961570e+00]])

In [0]:
train_x.drop(['lat_var_skew', 'lon_var_skew', 'velocity_var_skew',
       'velocity_var_covar', 'angle_var_mean', 'computed_angle_var_mean',
       'computed_angle_var_skew', 'velocity_first_counts',
       'velocity_second_counts'],axis=1,inplace=True)

In [0]:
#尝试lightgbm
import lightgbm as lgb

In [0]:
from sklearn.model_selection import GridSearchCV

In [0]:
#0.8847

In [0]:
parameters = {   
      'num_leaves': [90]
     
}
gbm = lgb.LGBMClassifier(objective = 'multi:softmax',
                         n_estimators=900,
                         max_depth = 11,
                         num_leaves = 90,
                         learning_rate = 0.17,
                         feature_fraction = 0.7,
                         min_child_samples=5,
                         min_child_weight=0.001,
                         bagging_fraction = 1,
                         bagging_freq = 0,
                         reg_alpha = 0.015,
                         reg_lambda = 0,
                         cat_smooth = 0,
                         device= 'gpu',
                        
                         gpu_platform_id= 1,
                         gpu_device_id= 0,
                         class_weight='balanced',
                         verbose=10,
                         n_jobs=-1
                        )
gsearch = GridSearchCV(estimator=gbm, param_grid=parameters, scoring='f1_macro', cv=5, verbose=10, n_jobs=1)
gsearch.fit(train_x, train_y)
print('参数的最佳取值:{0}'.format(gsearch.best_params_))
print('最佳模型得分:{0}'.format(gsearch.best_score_))
print(gsearch.cv_results_['mean_test_score'])
print(gsearch.cv_results_['params'])

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] num_leaves=90 ...................................................
[CV] ....................... num_leaves=90, score=0.893, total= 5.4min
[CV] num_leaves=90 ...................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  5.4min remaining:    0.0s


[CV] ....................... num_leaves=90, score=0.894, total= 5.4min
[CV] num_leaves=90 ...................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 10.8min remaining:    0.0s


In [0]:
len(train_x.columns)

225

In [0]:
!pip install shap

     |████████████████████████████████| 276kB 2.6MB/s 
  Created wheel for shap: filename=shap-0.35.0-cp36-cp36m-linux_x86_64.whl size=394154 sha256=3dd48b75a87a174ad7afd9c0c57184331f161c293b08fbf86f4f8c748bd8837f
  Stored in directory: /root/.cache/pip/wheels/e7/f7/0f/b57055080cf8894906b3bd3616d2fc2bfd0b12d5161bcb24ac
Successfully built shap


In [0]:
import shap

In [0]:
lgb1 = lgb.LGBMClassifier(objective = 'multi:softmax',
                         n_estimators=900,
                         max_depth = 11,
                         num_leaves = 90,
                         learning_rate = 0.27,
                         feature_fraction = 0.7,
                         min_child_samples=5,
                         min_child_weight=0.001,
                         bagging_fraction = 1,
                         bagging_freq = 0,
                         reg_alpha = 0.015,
                         reg_lambda = 0,
                         cat_smooth = 0,
                         #device= 'gpu',
                        
                         #gpu_platform_id= 1,
                         #gpu_device_id= 0,
                         class_weight='balanced'
                        )


In [0]:
lgb1.fit(train_x,train_y)

LGBMClassifier(bagging_fraction=1, bagging_freq=0, boosting_type='gbdt',
               cat_smooth=0, class_weight='balanced', colsample_bytree=1.0,
               feature_fraction=0.7, importance_type='split',
               learning_rate=0.27, max_depth=11, min_child_samples=5,
               min_child_weight=0.001, min_split_gain=0.0, n_estimators=900,
               n_jobs=-1, num_leaves=90, objective='multi:softmax',
               random_state=None, reg_alpha=0.015, reg_lambda=0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [0]:
shap.initjs()

In [0]:
explainer = shap.TreeExplainer(lgb1)
shap_values = explainer.shap_values(train_x) 

Setting feature_perturbation = "tree_path_dependent" because no background data was given.


In [0]:
shap.force_plot(explainer.expected_value, shap_values, train_x)

In [0]:
lgb1.feature_importances_

array([2453, 1999, 1211, 1398, 1853, 1515,  528, 1673,  535, 1125, 1401,
        473,  146, 2234, 1932, 1193, 1312, 1591, 1376,  516, 1475,  449,
       1385, 1396,  456,  164,  671,  915, 1392, 1613, 1921, 2298,  643,
       1604, 1305,  842, 1669, 1188, 1369,  573, 1620, 1334, 1466, 1422,
       1735, 1126, 1104, 1288,  681, 1118,  898, 1061,  980,  627,  796,
        807, 1126, 1135,  767, 1112,  603,  922,  711,  572,  648,  725,
        559,  635,  717, 1078,  989,  689,  977,  591,  947,  579,  461,
        501,  423,  699,  421,  476,  666,  941,  431,  543,  342,  628,
        584,  381,  393, 1246,  947, 2114, 1430, 2705, 2303,  980, 3319,
       1116, 2278, 2156, 1575, 1902, 1663, 1568, 2242, 1209, 2090, 1634,
       1338, 2372,  767, 1635, 1513, 1239, 1483,  861,  924, 1397,  675,
       1254, 1091,  713, 1182,  993,  864,  703,  517,  593,  785,  717,
       1234,  579, 1219,  998,  626, 1485,  361, 1062,  986,  791,  942,
        405,  641,  407,  495,  669,  873,  484,  3

In [0]:
train_x.columns[lgb1.feature_importances_>1500]

Index(['lat_min', 'lat_max', 'lat_skew', 'lat_kurt', 'lat_covar', 'lon_min',
       'lon_max', 'lon_skew', '速度_std', '速度_skew', '速度_kurt', '速度_covar',
       '速度_fengzhi', '方向_max', '方向_kurt', 'velocity_var_mean',
       'velocity_var_skew', 'velocity_var_kurt', 'velocity_var_covar',
       'velocity_var_boxing', 'velocity_var_fengzhi', 'velocity_var_maichong',
       'velocity_var_yudu', 'angle_var_min', 'angle_var_max', 'angle_var_mean',
       'angle_var_skew', 'angle_var_kurt', 'angle_var_covar',
       'angle_var_boxing', 'angle_var_fengzhi', 'lon_quantile_0.25',
       'velocity_var_quantile_0.25', 'angle_var_quantile_0.25',
       'velocity_var_quantile_0.75', 'angle_var_quantile_0.75', 'latlon_cov',
       'angle_var_computed_angle_var_cov', 'angle_var_velocity_cov',
       'angle_var_velocity_var_cov', 'hour', 'lat_fengzhi_lat_max_mul',
       'lon_max_lat_fengzhi_mul'],
      dtype='object')

In [0]:
train_x_dupl = train_x[['lat_min', 'lat_max', 'lat_mean', 'lat_std', 'lat_skew', 'lat_kurt',
       'lat_covar', 'lat_boxing', 'lat_fengzhi', 'lon_min', 'lon_max',
       'lon_mean', 'lon_skew', 'lon_covar', 'lon_boxing', '速度_std', '速度_skew',
       '速度_kurt', '速度_covar', '速度_fengzhi', '方向_max', '方向_std', '方向_kurt',
       'velocity_var_mean', 'velocity_var_skew', 'velocity_var_kurt',
       'velocity_var_covar', 'velocity_var_boxing', 'velocity_var_fengzhi',
       'velocity_var_maichong', 'velocity_var_yudu', 'angle_var_min',
       'angle_var_max', 'angle_var_mean', 'angle_var_skew', 'angle_var_kurt',
       'angle_var_covar', 'angle_var_boxing', 'angle_var_fengzhi',
       'lat_quantile_0.25', 'lon_quantile_0.25', 'velocity_var_quantile_0.25',
       'angle_var_quantile_0.25', 'lat_quantile_0.75', 'lon_quantile_0.75',
       '速度_quantile_0.75', 'velocity_var_quantile_0.75',
       'angle_var_quantile_0.75', 'latlon_cov',
       'angle_var_computed_angle_var_cov', 'angle_var_velocity_cov',
       'angle_var_velocity_var_cov', 'hour']]

In [0]:
def multiply_cols(data):
  data_ = data.copy()
  columns = data_.columns
  result = pd.DataFrame()
  columns_ = columns
  for col in columns:
    #删除掉该列标记
    columns_ = columns_.drop(col)
    #清除掉该列
    new_col = data_.pop(col)
    #该列与剩余的列组合
    new_cols = [x+'_'+col for x in columns_]
    result_ = data_.apply(lambda x : x*new_col.values)
    result_.columns = new_cols
    result = pd.concat([result,result_],axis=1)
  result.index = data_.index
  return result

In [0]:
train_x_dupl_ = multiply_cols(train_x_dupl)

In [0]:
train_x_dupl_

,lat_max_lat_min,lat_mean_lat_min,lat_std_lat_min,lat_skew_lat_min,lat_kurt_lat_min,lat_covar_lat_min,lat_boxing_lat_min,lat_fengzhi_lat_min,lon_min_lat_min,lon_max_lat_min,lon_mean_lat_min,lon_skew_lat_min,lon_covar_lat_min,lon_boxing_lat_min,速度_std_lat_min,速度_skew_lat_min,速度_kurt_lat_min,速度_covar_lat_min,速度_fengzhi_lat_min,方向_max_lat_min,方向_std_lat_min,方向_kurt_lat_min,velocity_var_mean_lat_min,velocity_var_skew_lat_min,velocity_var_kurt_lat_min,velocity_var_covar_lat_min,velocity_var_boxing_lat_min,velocity_var_fengzhi_lat_min,velocity_var_maichong_lat_min,velocity_var_yudu_lat_min,angle_var_min_lat_min,angle_var_max_lat_min,angle_var_mean_lat_min,angle_var_skew_lat_min,angle_var_kurt_lat_min,angle_var_covar_lat_min,angle_var_boxing_lat_min,angle_var_fengzhi_lat_min,lat_quantile_0.25_lat_min,lon_quantile_0.25_lat_min,...,angle_var_computed_angle_var_cov_lat_quantile_0.75,angle_var_velocity_cov_lat_quantile_0.75,angle_var_velocity_var_cov_lat_quantile_0.75,hour_lat_quantile_0.75,速度_quantile_0.75_lon_quantile_0.75,velocity_var_quantile_0.75_lon_quantile_0.75,angle_var_quantile_0.75_lon_quantile_0.75,latlon_cov_lon_quantile_0.75,angle_var_computed_angle_var_cov_lon_quantile_0.75,angle_var_velocity_cov_lon_quantile_0.75,angle_var_velocity_var_cov_lon_quantile_0.75,hour_lon_quantile_0.75,velocity_var_quantile_0.75_速度_quantile_0.75,angle_var_quantile_0.75_速度_quantile_0.75,latlon_cov_速度_quantile_0.75,angle_var_computed_angle_var_cov_速度_quantile_0.75,angle_var_velocity_cov_速度_quantile_0.75,angle_var_velocity_var_cov_速度_quantile_0.75,hour_速度_quantile_0.75,angle_var_quantile_0.75_velocity_var_quantile_0.75,latlon_cov_velocity_var_quantile_0.75,angle_var_computed_angle_var_cov_velocity_var_quantile_0.75,angle_var_velocity_cov_velocity_var_quantile_0.75,angle_var_velocity_var_cov_velocity_var_quantile_0.75,hour_velocity_var_quantile_0.75,latlon_cov_angle_var_quantile_0.75,angle_var_computed_angle_var_cov_angle_var_quantile_0.75,angle_var_velocity_cov_angle_var_quantile_0.75,angle_var_velocity_var_cov_angle_var_quantile_0.75,hour_angle_var_quantile_0.75,angle_var_computed_angle_var_cov_latlon_cov,angle_var_velocity_cov_latlon_cov,angle_var_velocity_var_cov_latlon_cov,hour_latlon_cov,angle_var_velocity_cov_angle_var_computed_angle_var_cov,angle_var_velocity_var_cov_angle_var_computed_angle_var_cov,hour_angle_var_computed_angle_var_cov,angle_var_velocity_var_cov_angle_var_velocity_cov,hour_angle_var_velocity_cov,hour_angle_var_velocity_var_cov
0,542.372045,540.994951,1.020305e+00,-11.754306,-21.951951,4.378281e-02,23.214942,23.274042,2729.341120,2732.289425,2730.915542,-1.385925,7.454503e-03,23.214981,15.815973,-33.085892,41.124559,4.864711,28.305090,8125.250,1832.370298,148.926581,-0.108666,-33.438275,134.474694,-1424.755430,36.853263,44.582971,70.812911,93.202372,-3366.17500,2809.0150,-22.474096,-18.581442,206.592083,-969.320485,50.732532,69.480355,540.184031,2730.246505,...,-127.089424,54.437101,158.584501,52.516125,444.789765,14.120310,1000.188625,-1.900460e-01,-640.711092,274.440258,799.490983,264.755813,0.453600,32.130000,-6.105026e-03,-20.582165,8.816103,25.682801,8.505000,1.02000,-1.938103e-04,-0.653402,0.279876,0.815327,0.270000,-0.013728,-46.282646,19.824569,57.752330,19.125000,0.008794,-0.003767,-0.010974,-3.633944e-03,-12.699426,-36.995580,-12.251289,15.846575,5.247680,15.287381
1,543.119661,541.461457,9.098935e-01,9.698899,-30.624472,3.906502e-02,23.246933,23.318150,2732.917320,2736.357876,2735.081269,-14.189830,8.921587e-03,23.246982,26.246643,-12.535378,75.610141,6.921750,43.775371,7625.016,1849.521166,7.415683,-0.474832,31.648419,191.699805,-1015.124829,45.874640,84.839403,167.453853,249.172596,-1168.16175,2464.1820,84.950473,46.104580,165.832094,175.780054,42.282147,88.408549,540.614797,2734.277269,...,-181.864221,200.481589,403.816459,149.675167,501.655756,18.830480,912.101375,-2.064795e-01,-917.589434,1011.522698,2037.441524,755.180708,0.682000,33.034375,-7.478249e-03,-33.233141,36.635204,73.791806,27.351042,1.24000,-2.8

In [0]:
lgb2 = lgb.LGBMClassifier()

In [0]:
lgb2.fit(train_x_dupl_,train_y)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [0]:
lgb2.feature_importances_[lgb2.feature_importances_>100]

array([138, 152, 113, 168, 177, 109, 254, 167, 218, 108, 201, 116, 109,
       104])

In [0]:
train_x_dupl_

,lat_max_lat_min,lat_mean_lat_min,lat_std_lat_min,lat_skew_lat_min,lat_kurt_lat_min,lat_covar_lat_min,lat_boxing_lat_min,lat_fengzhi_lat_min,lon_min_lat_min,lon_max_lat_min,lon_mean_lat_min,lon_skew_lat_min,lon_covar_lat_min,lon_boxing_lat_min,速度_std_lat_min,速度_skew_lat_min,速度_kurt_lat_min,速度_covar_lat_min,速度_fengzhi_lat_min,方向_max_lat_min,方向_std_lat_min,方向_kurt_lat_min,velocity_var_mean_lat_min,velocity_var_skew_lat_min,velocity_var_kurt_lat_min,velocity_var_covar_lat_min,velocity_var_boxing_lat_min,velocity_var_fengzhi_lat_min,velocity_var_maichong_lat_min,velocity_var_yudu_lat_min,angle_var_min_lat_min,angle_var_max_lat_min,angle_var_mean_lat_min,angle_var_skew_lat_min,angle_var_kurt_lat_min,angle_var_covar_lat_min,angle_var_boxing_lat_min,angle_var_fengzhi_lat_min,lat_quantile_0.25_lat_min,lon_quantile_0.25_lat_min,...,angle_var_computed_angle_var_cov_lat_quantile_0.75,angle_var_velocity_cov_lat_quantile_0.75,angle_var_velocity_var_cov_lat_quantile_0.75,hour_lat_quantile_0.75,速度_quantile_0.75_lon_quantile_0.75,velocity_var_quantile_0.75_lon_quantile_0.75,angle_var_quantile_0.75_lon_quantile_0.75,latlon_cov_lon_quantile_0.75,angle_var_computed_angle_var_cov_lon_quantile_0.75,angle_var_velocity_cov_lon_quantile_0.75,angle_var_velocity_var_cov_lon_quantile_0.75,hour_lon_quantile_0.75,velocity_var_quantile_0.75_速度_quantile_0.75,angle_var_quantile_0.75_速度_quantile_0.75,latlon_cov_速度_quantile_0.75,angle_var_computed_angle_var_cov_速度_quantile_0.75,angle_var_velocity_cov_速度_quantile_0.75,angle_var_velocity_var_cov_速度_quantile_0.75,hour_速度_quantile_0.75,angle_var_quantile_0.75_velocity_var_quantile_0.75,latlon_cov_velocity_var_quantile_0.75,angle_var_computed_angle_var_cov_velocity_var_quantile_0.75,angle_var_velocity_cov_velocity_var_quantile_0.75,angle_var_velocity_var_cov_velocity_var_quantile_0.75,hour_velocity_var_quantile_0.75,latlon_cov_angle_var_quantile_0.75,angle_var_computed_angle_var_cov_angle_var_quantile_0.75,angle_var_velocity_cov_angle_var_quantile_0.75,angle_var_velocity_var_cov_angle_var_quantile_0.75,hour_angle_var_quantile_0.75,angle_var_computed_angle_var_cov_latlon_cov,angle_var_velocity_cov_latlon_cov,angle_var_velocity_var_cov_latlon_cov,hour_latlon_cov,angle_var_velocity_cov_angle_var_computed_angle_var_cov,angle_var_velocity_var_cov_angle_var_computed_angle_var_cov,hour_angle_var_computed_angle_var_cov,angle_var_velocity_var_cov_angle_var_velocity_cov,hour_angle_var_velocity_cov,hour_angle_var_velocity_var_cov
0,542.372045,540.994951,1.020305e+00,-11.754306,-21.951951,4.378281e-02,23.214942,23.274042,2729.341120,2732.289425,2730.915542,-1.385925,7.454503e-03,23.214981,15.815973,-33.085892,41.124559,4.864711,28.305090,8125.250,1832.370298,148.926581,-0.108666,-33.438275,134.474694,-1424.755430,36.853263,44.582971,70.812911,93.202372,-3366.17500,2809.0150,-22.474096,-18.581442,206.592083,-969.320485,50.732532,69.480355,540.184031,2730.246505,...,-127.089424,54.437101,158.584501,52.516125,444.789765,14.120310,1000.188625,-1.900460e-01,-640.711092,274.440258,799.490983,264.755813,0.453600,32.130000,-6.105026e-03,-20.582165,8.816103,25.682801,8.505000,1.02000,-1.938103e-04,-0.653402,0.279876,0.815327,0.270000,-0.013728,-46.282646,19.824569,57.752330,19.125000,0.008794,-0.003767,-0.010974,-3.633944e-03,-12.699426,-36.995580,-12.251289,15.846575,5.247680,15.287381
1,543.119661,541.461457,9.098935e-01,9.698899,-30.624472,3.906502e-02,23.246933,23.318150,2732.917320,2736.357876,2735.081269,-14.189830,8.921587e-03,23.246982,26.246643,-12.535378,75.610141,6.921750,43.775371,7625.016,1849.521166,7.415683,-0.474832,31.648419,191.699805,-1015.124829,45.874640,84.839403,167.453853,249.172596,-1168.16175,2464.1820,84.950473,46.104580,165.832094,175.780054,42.282147,88.408549,540.614797,2734.277269,...,-181.864221,200.481589,403.816459,149.675167,501.655756,18.830480,912.101375,-2.064795e-01,-917.589434,1011.522698,2037.441524,755.180708,0.682000,33.034375,-7.478249e-03,-33.233141,36.635204,73.791806,27.351042,1.24000,-2.8

In [0]:
parameters = {   
      'num_leaves': [90]
     
}
gbm = lgb.LGBMClassifier(objective = 'multi:softmax',
                         n_estimators=900,
                         max_depth = 11,
                         num_leaves = 90,
                         learning_rate = 0.17,
                         feature_fraction = 0.7,
                         min_child_samples=5,
                         min_child_weight=0.001,
                         bagging_fraction = 1,
                         bagging_freq = 0,
                         reg_alpha = 0.015,
                         reg_lambda = 0,
                         cat_smooth = 0,
                         #device= 'gpu',
                        
                         #gpu_platform_id= 1,
                         #gpu_device_id= 0,
                         class_weight='balanced',
                         verbose=10,
                         n_jobs=-1
                        )
gsearch = GridSearchCV(estimator=gbm, param_grid=parameters, scoring='f1_macro', cv=5, verbose=10, n_jobs=1)
gsearch.fit(train_x_dupl_, train_y)
print('参数的最佳取值:{0}'.format(gsearch.best_params_))
print('最佳模型得分:{0}'.format(gsearch.best_score_))
print(gsearch.cv_results_['mean_test_score'])
print(gsearch.cv_results_['params'])

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] num_leaves=90 ...................................................
[CV] ....................... num_leaves=90, score=0.885, total=29.9min
[CV] num_leaves=90 ...................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 29.9min remaining:    0.0s


KeyboardInterrupt: ignored

In [0]:
train_x = train_x[train_x.columns[lgb1.feature_importances_>50]]

In [0]:
lgb1.feature_importances_[lgb1.feature_importances_>30]

array([3107, 3367, 1606, 1164, 1728, 1365,  624, 1997, 1522, 1633,  830,
        728,  718, 3198, 2876, 1457, 1098, 1517, 1273,  470, 1845, 1505,
       1658,  712,  747,  659,  576,  747, 1260, 1428, 1779, 2077,  533,
       1230, 1265, 1077, 1606, 1171, 1187,  520, 1232,  877, 1223, 1003,
       1387, 1016,  754,  890,  662,  903,  630,  826,  828,  534,  630,
        673, 1011,  990,  629,  947,  571,  784,  581,  502,  598,  659,
        480,  521,  611,  883,  870,  570,  827,  529,  809,  537,  381,
        456,  369,  595,  362,  422,  635,  787,  396,  523,  347,  579,
        506,  342,  388, 1144,  819, 1926, 1109, 2480, 2090,  980, 3083,
       1106, 1992, 1984, 1589, 1801, 1492, 1301, 1942,  795, 1844, 1333,
       1068, 2105,  773, 1387, 1341, 1078, 1282,  705,  811,  868,  533,
        897,  872,  610, 1007,  865,  756,  543,  463,  507,  694,  591,
        997,  337, 1101,  815,  586, 1302,  335,  901,  821,  689,  837,
        370,  486,  413,  439,  624,  797,  425,  2

In [0]:
train_x.columns[lgb1.feature_importances_>30]

Index(['lat_min', 'lat_max', 'lat_mean', 'lat_std', 'lat_skew', 'lat_kurt',
       'lat_range', 'lat_covar', 'lat_rms', 'lat_boxing',
       ...
       'circle_area', 'line_len', 'velocity_third_counts',
       'velocity_count_rate_1', 'velocity_count_rate_2',
       'velocity_count_rate_3', 'velocity_count_rate_4',
       'velocity_count_rate_5', 'velocity_count_rate_6', 'hour'],
      dtype='object', length=298)

In [0]:
train_prob = lgb1.predict_proba(train_x)

In [0]:
train_prob = pd.concat([train_id,pd.DataFrame(train_prob)],axis=1)

In [0]:
train_prob = train_prob.groupby('id').mean()

In [0]:
train_prob=train_prob.sort_values(by='id')

In [0]:
train_y_ = pd.concat([train_id,train_y],axis=1)

In [0]:
train_y_ = train_y_.groupby('id').head(1)

In [0]:
train_y_ = train_y_.set_index('id')

In [0]:
train_y_ = train_y_.sort_values(by='id')

In [0]:
test_prob = lgb1.predict_proba(test_x)

In [0]:
test_prob = pd.concat([test_id,pd.DataFrame(test_prob)],axis=1)

In [0]:
test_prob = test_prob.groupby('id').mean()

In [0]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import f1_score

In [0]:
lr = LogisticRegressionCV(cv=5,class_weight='balanced',scoring='f1_macro',verbose=10)

In [0]:
lr.fit(train_prob,list(train_y_['type']))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    1.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    1.2s finished


LogisticRegressionCV(Cs=10, class_weight='balanced', cv=5, dual=False,
                     fit_intercept=True, intercept_scaling=1.0, l1_ratios=None,
                     max_iter=100, multi_class='auto', n_jobs=None,
                     penalty='l2', random_state=None, refit=True,
                     scoring='f1_macro', solver='lbfgs', tol=0.0001,
                     verbose=10)

In [0]:
test_y_ = lr.predict(test_prob)

In [0]:
test_y_ = pd.concat([pd.Series(list(test_prob.index)),pd.Series(test_y_)],axis=1)

In [0]:
test_y_.set_index(0,inplace=True)

In [0]:
test_y_.to_csv('./drive2/Colab Notebooks/result1.csv',header=None,encoding='utf-8')

In [0]:
lgb1.feature_importances_

array([2285, 1991, 1927, 1489, 1407, 1366, 1959, 2693, 1472, 1353, 1720,
       1330, 1637, 1370, 1367, 1730, 1360, 1680, 1557, 1907, 1562, 1284,
       1404, 1504, 1489, 1681, 1839, 1469, 1982, 1693, 1313, 1399, 1245,
       1619, 1578, 1711, 1690, 1514, 1674, 1484, 1390, 1930, 1877, 1567,
       1454, 1424, 1889, 1879, 1799, 1866, 1782, 2196, 2263, 1346, 1636,
       2460, 1399, 1678, 1664])

In [0]:
len(lgb1.feature_importances_[lgb1.feature_importances_>1400])

46

In [0]:
train_x.columns[lgb1.feature_importances_<=1400]

Index(['x_rms', 'y_kurt', 'y_yudu', '速度_skew', '速度_kurt', '速度_fengzhi',
       'y_var_min', 'velocity_var_boxing', 'velocity_var_fengzhi',
       'velocity_var_yudu', 'computed_angle_min', 'angle_var_quantile_0.25',
       'angle_var_quantile_0.75'],
      dtype='object')

In [0]:
train_x = train_x[train_x.columns[lgb1.feature_importances_>1400]]

In [0]:
train_x.columns

Index(['x_min', 'x_max', 'x_std', 'x_skew', 'x_kurt', 'y_min', 'y_max',
       'y_skew', 'y_covar', '速度_std', '速度_boxing', '方向_std', '方向_kurt',
       '方向_yudu', 'x_var_skew', 'y_var_max', 'y_var_skew', 'y_var_covar',
       'velocity_var_mean', 'velocity_var_skew', 'velocity_var_kurt',
       'velocity_var_covar', 'velocity_var_rms', 'angle_var_min',
       'angle_var_max', 'angle_var_mean', 'angle_var_skew', 'angle_var_kurt',
       'angle_var_covar', 'angle_var_boxing', 'computed_angle_max',
       'computed_angle_mean', 'computed_angle_covar', 'haversine_max',
       'dummy_manhattan_distance_kurt', 'bearing_min', 'bearing_max',
       'bearing_mean', 'bearing_skew', 'bearing_covar', 'x_quantile_0.25',
       'y_quantile_0.25', 'x_quantile_0.75', 'y_quantile_0.75', 'xy_cov',
       'velocity_count_rate_3'],
      dtype='object')

In [0]:
from sklearn.ensemble import  StackingClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier

In [0]:
import lightgbm as lgb
from catboost import CatBoostClassifier
from sklearn.utils import class_weight

In [0]:
cw = list(class_weight.compute_class_weight('balanced',np.unique(train_y),train_y))
lr = LogisticRegressionCV(cv=5,class_weight='balanced',scoring='f1_macro',verbose=10)  
clf1 = lgb.LGBMClassifier(objective = 'multi:softmax',
                         n_estimators=900,
                         max_depth = 11,
                         num_leaves = 90,
                         learning_rate = 0.27,
                         feature_fraction = 0.7,
                         min_child_samples=5,
                         min_child_weight=0.001,
                         bagging_fraction = 1,
                         bagging_freq = 0,
                         reg_alpha = 0.015,
                         reg_lambda = 0,
                         cat_smooth = 0,
                         #device= 'gpu',
                         #gpu_platform_id= 1,
                         #gpu_device_id= 0,
                         class_weight='balanced'
                        )
clf2 = RandomForestClassifier(n_estimators=1000,class_weight='balanced')
clf3 = CatBoostClassifier(iterations=2000,verbose=400,early_stopping_rounds=200,#task_type='GPU',
                          border_count=254,
                    loss_function='MultiClass',
                         class_weights=cw,depth=8,l2_leaf_reg=0.06,random_strength=0.01)

clf = StackingClassifier(estimators=[('lgb',clf1),('rf',clf2),('catboost',clf3)],cv=5,final_estimator=lr,
                         stack_method='predict_proba',verbose=10,n_jobs=1)

clf.fit(train_x, train_y)

NameError: ignored

In [0]:
!ls

In [0]:
# 对未知数据进行训练
y_test_labels = clf.predict(test_x)  # 标签预测
y_test_proba = clf.predict_proba(test_x)  # 概率预测

In [0]:
y_test_prob = pd.concat([test_id,pd.DataFrame(y_test_proba)],axis=1)

In [0]:
y_test = y_test_prob.groupby('id').mean()

In [0]:
#这个文件数据量有点少 被写的代码里给过滤掉了 单独预测补充进去
data9128 = pd.read_csv('hy_round1_testB_20200221/9128.csv')

In [0]:
data9128 = preprocessing(data9128,9128)

In [0]:
data9128 = pd.DataFrame.from_dict((data9128,))

In [0]:
data9128 = data9128.set_index('id')

In [0]:
result9128 = clf.predict(data9128)

In [0]:
result9128 = pd.concat([pd.Series([9128]),pd.Series(result9128)],axis=1)

In [0]:
result9128.set_index(0,inplace=True)

In [0]:
len(test_id.unique())

In [0]:
clf.classes_

array(['刺网', '围网', '拖网'], dtype=object)

In [0]:
result = [clf.classes_[i] for i in np.argmax(y_test.values,axis=1)]

In [0]:
result = pd.concat([pd.Series(y_test.index),pd.Series(result)],axis=1)

In [0]:
result.set_index('id',inplace=True)

In [0]:
result9128.columns = [0]

In [0]:
result = pd.concat([result,result9128],axis=0)

In [0]:
result.to_csv('./drive2/Colab Notebooks/result1.csv',header=None,encoding='utf-8')

In [0]:
result[0].value_counts()/2000

拖网    0.5865
围网    0.2690
刺网    0.1445
Name: 0, dtype: float64